In [1]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [2]:
# importing required libraries
import pandas as pd
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report
from sklearn import model_selection, preprocessing, linear_model, naive_bayes, metrics, svm
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn import decomposition, ensemble

In [3]:
train_data= pd.read_csv('/content/drive/My Drive/Hate Speech/hindi_dataset.tsv', sep = '\t', encoding="latin-1")  #Load train data
test_data = pd.read_csv('/content/drive/My Drive/Hate Speech/hasoc2019_hi_test_gold_2919.tsv', sep = '\t', encoding="latin-1") #load test data

In [4]:
train_data=train_data[train_data.task_2!="NONE"]       #drop none
test_data=test_data[test_data.task_2!="NONE"]

In [5]:
print('Shape of training data :',train_data.shape)
print('Shape of testing data :',test_data.shape)

Shape of training data : (2469, 5)
Shape of testing data : (605, 5)


In [6]:
train_x = train_data['text']
train_y = train_data['task_2']

In [7]:
test_x = test_data['text']
test_y = test_data['task_2']

In [8]:
model = XGBClassifier() #for count vectorization
model1= XGBClassifier() #for tfidf

In [9]:
count_vect = CountVectorizer(analyzer='word', token_pattern=r'\w{1,}')  #count vectorization
count_vect.fit(train_data['text'])      

x_train_count =  count_vect.transform(train_x)
x_test_count =  count_vect.transform(test_x)

In [10]:
tfidf_vect= TfidfVectorizer(analyzer='word', token_pattern=r'\w{1,}')      #tfidf
tfidf_vect.fit(train_data['text'])

x_train_tfidf =  tfidf_vect.transform(train_x)
x_test_tfidf =  tfidf_vect.transform(test_x)

In [11]:
encoder = preprocessing.LabelEncoder()    #encoding y
train_y = encoder.fit_transform(train_y)
test_y = encoder.fit_transform(test_y)

In [12]:
model.fit(x_train_count.tocsc(),train_y)  #count vectorization training

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0,
              learning_rate=0.1, max_delta_step=0, max_depth=3,
              min_child_weight=1, missing=None, n_estimators=100, n_jobs=1,
              nthread=None, objective='multi:softprob', random_state=0,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
              silent=None, subsample=1, verbosity=1)

In [13]:
model1.fit(x_train_tfidf.tocsc(),train_y)   #tfidf training

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0,
              learning_rate=0.1, max_delta_step=0, max_depth=3,
              min_child_weight=1, missing=None, n_estimators=100, n_jobs=1,
              nthread=None, objective='multi:softprob', random_state=0,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
              silent=None, subsample=1, verbosity=1)

In [14]:
y_pred_count=model.predict(x_test_count.tocsc())  #tocsc convert it to sparse matrix

In [15]:
y_pred_tfidf=model.predict(x_test_tfidf.tocsc())

In [16]:
print('Model = Xgboost for Count Vectorization\n')
print(classification_report(y_pred_count, test_y))

Model = Xgboost for Count Vectorization

              precision    recall  f1-score   support

           0       0.18      0.42      0.26        84
           1       0.11      0.44      0.18        50
           2       0.88      0.41      0.56       471

    accuracy                           0.41       605
   macro avg       0.39      0.42      0.33       605
weighted avg       0.72      0.41      0.48       605



In [17]:
print('Model = Xgboost for Tfidf\n')
print(classification_report(y_pred_tfidf, test_y))

Model = Xgboost for Tfidf

              precision    recall  f1-score   support

           0       0.03      0.38      0.06        16
           1       0.01      0.67      0.02         3
           2       0.99      0.37      0.53       586

    accuracy                           0.37       605
   macro avg       0.34      0.47      0.20       605
weighted avg       0.96      0.37      0.52       605

